In [2]:
import spacy

# Download the 'en_core_web_sm' model
spacy.cli.download("en_core_web_sm")

# Load the downloaded model
nlp = spacy.load("en_core_web_sm")


✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')


In [3]:
import re
import spacy
import numpy as np
import pandas as pd
import en_core_web_sm
import tensorflow as tf
from nltk.stem import WordNetLemmatizer
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.models import Sequential
from spacy.lang.en.stop_words import STOP_WORDS
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.layers.experimental.preprocessing import TextVectorization
from tensorflow.keras.layers import Embedding, LSTM, Dense, Bidirectional, Dropout
nlp = en_core_web_sm.load()
lemmatizer = WordNetLemmatizer()
stopwords = STOP_WORDS
EMOJI_PATTERN = re.compile(
    "["
    u"U0001F600-U0001F64F"  # emoticons
    u"U0001F300-U0001F5FF"  # symbols & pictographs
    u"U0001F680-U0001F6FF"  # transport & map symbols
    u"U0001F1E0-U0001F1FF"  # flags (iOS)
    u"U00002702-U000027B0"
    u"U000024C2-U0001F251"
    "]+", flags=re.UNICODE
)
FILTERS = '!"#$%&()*+,-/:;?@[\]^_`{|}~tn'
HTML_TAG_PATTERN = re.compile(r']*>')
NUMBERING_PATTERN = re.compile('d+(?:st|[nr]d|th)')
DISABLE_PIPELINES = ["tok2vec", "parser", "ner", "textcat", "custom", "lemmatizer"]

In [4]:
import pandas as pd
train_file_path = 'C:/Users/Ajay/Downloads/train_150k.txt'
test_file_path = 'C:/Users/Ajay/Downloads/test_62k.txt'
def load_data(file_path):
    with open(file_path, 'r', encoding='utf-8') as file:
        data = file.readlines()
    return [line.strip().split('\t') for line in data]
train_data = load_data(train_file_path)
test_data = load_data(test_file_path)
df = pd.DataFrame(train_data, columns=['sentiment', 'text'])
test_df = pd.DataFrame(test_data, columns=['sentiment', 'text'])
df.head()

,sentiment,text
0,0,Starting back at work today Looks like it'l...
1,1,Sugar levels dropping... munchies setting in. ...
2,1,@karineb22 yeah!!! have a great summer break!
3,1,hannah montana was very good. now going to re...
4,1,"@Mayra326 aww, have fun! I just had my 3D las..."


In [8]:
def initial_preprocessing(text):


    tag_removed_text = HTML_TAG_PATTERN.sub('', text)

    emoji_removed_text = EMOJI_PATTERN.sub(r'', tag_removed_text)

    numberings_removed_text =  NUMBERING_PATTERN.sub('', emoji_removed_text)

    extra_chars_removed_text = re.sub(

        r"(.)1{2,}",  r'11', numberings_removed_text

    )

    return extra_chars_removed_text

def preprocess_text(doc):



    tokens = [

        token

        for token in doc

        if not token.is_space and

           not token.like_email and

           not token.like_url and

           not token.is_stop and

           not token.is_punct and

           not token.like_num

    ]



    translation_table = str.maketrans('', '', FILTERS)

    translated_tokens = [

        token.text.lower().translate(translation_table)

        for token in tokens

    ]



    lemmatized_tokens = [

        lemmatizer.lemmatize(token)

        for token in translated_tokens

        if len(token) > 1

    ]

    return lemmatized_tokens

labels = df['sentiment']

labels_test = test_df['sentiment']


column = 'text'

not_null_data = df[df[column].notnull()]

not_null_data[column] = not_null_data[column].apply(initial_preprocessing)

texts = [

    preprocess_text(doc)

    for doc in nlp.pipe(not_null_data[column], disable=DISABLE_PIPELINES)

]

In [9]:
#for train 
tokenizer = Tokenizer(
    filters=FILTERS,
    lower=True
)
padding = 'post'
tokenizer.fit_on_texts(texts)
vocab_size = len(tokenizer.word_index) + 1
sequences = []
max_sequence_len = 0
for text in texts:
    # convert texts to sequence
    txt_to_seq = tokenizer.texts_to_sequences([text])[0]
    sequences.append(txt_to_seq)
    # find max_sequence_len for padding
    txt_to_seq_len = len(txt_to_seq)
    if txt_to_seq_len > max_sequence_len:
        max_sequence_len = txt_to_seq_len
# post padding
padded_sequences = pad_sequences(
    sequences, 
    maxlen=max_sequence_len, 
    padding=padding
)

In [11]:
not_null_data = test_df[test_df[column].notnull()]

not_null_data[column] = not_null_data[column].apply(initial_preprocessing)

texts_test = [

    preprocess_text(doc)

    for doc in nlp.pipe(not_null_data[column], disable=DISABLE_PIPELINES)

]

In [12]:
tokenizer = Tokenizer(
    filters=FILTERS,
    lower=True
)
padding = 'post'
tokenizer.fit_on_texts(texts)
vocab_size = len(tokenizer.word_index) + 1
sequences = []
max_sequence_len = 0
for text in texts_test:
    # convert texts to sequence
    txt_to_seq = tokenizer.texts_to_sequences([text])[0]
    sequences.append(txt_to_seq)
    # find max_sequence_len for padding
    txt_to_seq_len = len(txt_to_seq)
    if txt_to_seq_len > max_sequence_len:
        max_sequence_len = txt_to_seq_len
# post padding
test_padded_sequences = pad_sequences(
    sequences, 
    maxlen=max_sequence_len, 
    padding=padding
)

In [15]:
max_sequence_len = 103  

model = Sequential()
model.add(Embedding(vocab_size, 64, input_length=max_sequence_len))  
model.add(Bidirectional(LSTM(64, return_sequences=True)))
model.add(Dropout(0.2))
model.add(Bidirectional(LSTM(32)))
model.add(Dropout(0.2))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.1))
model.add(Dense(1, activation='sigmoid'))
adam = Adam(learning_rate=0.01)
model.compile(
    loss=tf.keras.losses.BinaryCrossentropy(from_logits=True), 
    optimizer=adam, 
    metrics=['accuracy']
)
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_1 (Embedding)     (None, 103, 64)           7525760   
                                                                 
 bidirectional_2 (Bidirecti  (None, 103, 128)          66048     
 onal)                                                           
                                                                 
 dropout_3 (Dropout)         (None, 103, 128)          0         
                                                                 
 bidirectional_3 (Bidirecti  (None, 64)                41216     
 onal)                                                           
                                                                 
 dropout_4 (Dropout)         (None, 64)                0         
                                                                 
 dense_2 (Dense)             (None, 64)               

In [17]:
labels = labels.map(lambda x: 1 if x=='1' else 0)
labels_test = labels_test.map(lambda x: 1 if x=='positive' else 0)


In [18]:
history = model.fit(
    padded_sequences, 
    labels.values, 
    epochs=10,
    verbose=1,
    batch_size=64
)

Epoch 1/10
2344/2344 [==============================] - 350s 148ms/step - loss: 0.5410 - accuracy: 0.7273
Epoch 2/10
2344/2344 [==============================] - 336s 143ms/step - loss: 0.3385 - accuracy: 0.8527
Epoch 3/10
2344/2344 [==============================] - 347s 148ms/step - loss: 0.2699 - accuracy: 0.8822
Epoch 4/10
2344/2344 [==============================] - 337s 144ms/step - loss: 0.2375 - accuracy: 0.8938
Epoch 5/10
2344/2344 [==============================] - 371s 158ms/step - loss: 0.2209 - accuracy: 0.9014
Epoch 6/10
2344/2344 [==============================] - 410s 175ms/step - loss: 0.2103 - accuracy: 0.9058
Epoch 7/10
2344/2344 [==============================] - 376s 161ms/step - loss: 0.2018 - accuracy: 0.9100
Epoch 8/10
2344/2344 [==============================] - 380s 162ms/step - loss: 0.1990 - accuracy: 0.9115
Epoch 9/10
2344/2344 [==============================] - 382s 163ms/step - loss: 0.1970 - accuracy: 0.9126
Epoch 10/10
2344/2344 [=======================